In [1]:
!pip install yfinance tensorflow==2.15 scikit-learn matplotlib gradio --quiet


ERROR: Could not find a version that satisfies the requirement tensorflow==2.15 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0, 2.19.1, 2.20.0rc0, 2.20.0)
ERROR: No matching distribution found for tensorflow==2.15


In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import gradio as gr

def predict_stock(symbol):
    try:
        df = yf.download(symbol, start="2020-01-01", end="2025-10-01", progress=False)
        if df is None or len(df) == 0:
            return f"Error: No data found for symbol '{symbol}'.", None

        data = df['Close'].values.reshape(-1, 1)
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(data)

        if len(scaled_data) < 61:
            return f"Error: Not enough data for '{symbol}'.", None

        X, y = [], []
        for i in range(60, len(scaled_data)):
            X.append(scaled_data[i-60:i, 0])
            y.append(scaled_data[i, 0])
        X, y = np.array(X), np.array(y)
        X = np.reshape(X, (X.shape[0], X.shape[1], 1))

        # LSTM model
        model = Sequential([
            LSTM(50, return_sequences=True, input_shape=(X.shape[1], 1)),
            LSTM(50, return_sequences=False),
            Dense(25),
            Dense(1)
        ])
        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(X, y, batch_size=32, epochs=1, verbose=0)

        # Predict next closing price
        last_60 = scaled_data[-60:]
        X_test = np.reshape(last_60, (1, 60, 1))
        pred = model.predict(X_test)
        pred_price = scaler.inverse_transform(pred)[0][0]

        # Plot figure
        fig, ax = plt.subplots(figsize=(8, 4))
        ax.plot(df.index, df['Close'], label='Historical Close', color='cyan')
        ax.set_title(f'{symbol} Stock Price Forecast')
        ax.set_xlabel('Date')
        ax.set_ylabel('Price (USD)')
        ax.legend()
        plt.tight_layout()

        return f"Predicted next closing price for {symbol}: ${pred_price:.2f}", fig

    except Exception as e:
        return f"Error: {str(e)}", None


In [ ]:
with gr.Blocks(theme=gr.themes.Monochrome()) as demo:
    gr.Markdown("## 📈 SmartTrade LSTM Stock Forecaster\nPredicts next closing price using an LSTM model trained on historical data.")
    stock = gr.Textbox(label="Enter Stock Symbol (e.g. AAPL, TSLA, MSFT)", value="AAPL")
    output_text = gr.Textbox(label="Forecast Summary")
    output_plot = gr.Plot(label="Forecast Chart")

    submit = gr.Button("Submit", variant="primary")
    submit.click(fn=predict_stock, inputs=stock, outputs=[output_text, output_plot])

demo.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://282934bb9552764426.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
